# Acid-Base Titration Curve Fitting with fit() API

**Domain:** Chemistry | **Level:** Intermediate

This example demonstrates acid-base titration curve analysis using NLSQ's
advanced fit() API and global optimization capabilities for robust pKa
and equivalence point determination.

**Key Concepts:**
- Henderson-Hasselbalch equation
- pKa determination from titration curves
- Equivalence point identification
- Buffer capacity analysis
- Global optimization for robust parameter estimation

In [ ]:
# Configure matplotlib for inline plotting
%matplotlib inline

In [ ]:
from pathlib import Path

import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np

from nlsq import GlobalOptimizationConfig, fit

# Set random seed
np.random.seed(42)

## Model Definitions

In [ ]:
def simplified_titration(V, pKa, Ve, pH0):
    """
    Simplified titration model for curve fitting.

    Parameters
    ----------
    V : array_like
        Volume of base added (mL)
    pKa : float
        Acid dissociation constant
    Ve : float
        Equivalence point volume (mL)
    pH0 : float
        Initial pH (at V=0)
    """
    V_safe = jnp.maximum(V, 0.01)
    Ve_safe = jnp.maximum(Ve, V_safe + 0.1)

    f = V_safe / Ve_safe
    f = jnp.clip(f, 0.01, 0.99)

    ratio = f / (1 - f)
    pH = pKa + jnp.log10(jnp.maximum(ratio, 1e-10))

    return pH


def buffer_capacity(pH, pKa, C_total):
    """
    Calculate buffer capacity as a function of pH.
    """
    H_plus = jnp.power(10.0, -pH)
    Ka = jnp.power(10.0, -pKa)

    numerator = H_plus * Ka
    denominator = jnp.power(H_plus + Ka, 2)

    beta = 2.303 * C_total * numerator / jnp.maximum(denominator, 1e-20)

    return beta

## Generate Synthetic Titration Data

In [ ]:
# Experimental parameters
V0_acid = 25.0  # mL of 0.1 M acetic acid
C_acid_true = 0.1  # M
C_base_true = 0.1  # M (NaOH)
pKa_true = 4.76  # Acetic acid
Ve_true = 25.0  # mL (equivalence point)

# Generate synthetic titration data
V_titrant = np.linspace(0.1, 40, 100)

# Calculate true pH values
pH_true = np.zeros_like(V_titrant)
for i, V in enumerate(V_titrant):
    if Ve_true > V:
        f = V / Ve_true
        if 0.001 < f < 0.999:
            pH_true[i] = pKa_true + np.log10(f / (1 - f))
        elif f <= 0.001:
            pH_true[i] = 2.9
        else:
            pH_true[i] = 8.0
    elif Ve_true == V:
        pH_true[i] = 8.72
    else:
        excess = (V - Ve_true) * C_base_true
        total_volume = V0_acid + V
        pOH = -np.log10(excess / total_volume)
        pH_true[i] = 14 - pOH

# Add noise
noise_level = 0.05
pH_measured = pH_true + np.random.normal(0, noise_level, size=pH_true.shape)
sigma_pH = np.full_like(pH_measured, noise_level)

print(f"True parameters:")
print(f"  pKa: {pKa_true}")
print(f"  Equivalence point: {Ve_true} mL")

## Monoprotic Acid Titration Analysis

In [ ]:
print("MONOPROTIC ACID TITRATION (Acetic Acid)")
print("=" * 60)

# Fit only the buffer region (5-35 mL) for better pKa estimation
mask_fit = (V_titrant >= 5) & (V_titrant <= 35)
V_fit = V_titrant[mask_fit]
pH_fit = pH_measured[mask_fit]
sigma_fit = sigma_pH[mask_fit]

# Initial guess and bounds
p0 = [4.5, 24.0, 3.0]  # pKa, Ve, pH0
bounds_lower = [3.0, 20.0, 2.5]
bounds_upper = [6.0, 30.0, 4.0]

# Method 1: fit() with 'robust' preset
print("\nMethod 1: fit() with 'robust' preset")
popt_robust, pcov_robust = fit(
    simplified_titration,
    V_fit,
    pH_fit,
    p0=p0,
    sigma=sigma_fit,
    bounds=(bounds_lower, bounds_upper),
    absolute_sigma=True,
    preset="robust",
)

pKa_fit, Ve_fit, pH0_fit = popt_robust
pKa_err, Ve_err, pH0_err = np.sqrt(np.diag(pcov_robust))

print(f"  pKa = {pKa_fit:.3f} +/- {pKa_err:.3f} (true: {pKa_true})")
print(f"  Ve = {Ve_fit:.2f} +/- {Ve_err:.2f} mL (true: {Ve_true})")

In [ ]:
# Method 2: fit() with 'global' preset
print("\nMethod 2: fit() with 'global' preset")
popt_global, pcov_global = fit(
    simplified_titration,
    V_fit,
    pH_fit,
    p0=p0,
    sigma=sigma_fit,
    bounds=(bounds_lower, bounds_upper),
    absolute_sigma=True,
    preset="global",
)

pKa_g, Ve_g, pH0_g = popt_global
perr_g = np.sqrt(np.diag(pcov_global))

print(f"  pKa = {pKa_g:.3f} +/- {perr_g[0]:.3f}")
print(f"  Ve = {Ve_g:.2f} +/- {perr_g[1]:.2f} mL")

In [ ]:
# Method 3: GlobalOptimizationConfig with custom settings
print("\nMethod 3: GlobalOptimizationConfig with custom settings")

global_config = GlobalOptimizationConfig(
    n_starts=15,
    sampler="lhs",
    center_on_p0=True,
    scale_factor=1.0,
)

popt_custom, pcov_custom = fit(
    simplified_titration,
    V_fit,
    pH_fit,
    p0=p0,
    sigma=sigma_fit,
    bounds=(bounds_lower, bounds_upper),
    absolute_sigma=True,
    multistart=True,
    n_starts=15,
    sampler="lhs",
)

pKa_c, Ve_c, pH0_c = popt_custom
perr_c = np.sqrt(np.diag(pcov_custom))

print(f"  pKa = {pKa_c:.3f} +/- {perr_c[0]:.3f}")
print(f"  Ve = {Ve_c:.2f} +/- {perr_c[1]:.2f} mL")

## Goodness of Fit and Inflection Point Analysis

In [ ]:
# Use robust preset results for analysis
pKa_fit, Ve_fit, pH0_fit = popt_robust
perr = np.sqrt(np.diag(pcov_robust))
pKa_err, Ve_err, pH0_err = perr

# Goodness of fit
pH_fitted_curve = simplified_titration(V_fit, *popt_robust)
residuals = pH_fit - pH_fitted_curve
chi_squared = np.sum((residuals / sigma_fit) ** 2)
dof = len(pH_fit) - len(popt_robust)
reduced_chi_squared = chi_squared / dof
rmse = np.sqrt(np.mean(residuals**2))

print("\n" + "=" * 60)
print("FITTED PARAMETERS (Robust Preset)")
print("=" * 60)
print(f"  pKa = {pKa_fit:.3f} +/- {pKa_err:.3f}")
print(f"  Equivalence point = {Ve_fit:.2f} +/- {Ve_err:.2f} mL")

print("\nGoodness of Fit:")
print(f"  chi^2/dof = {reduced_chi_squared:.3f} (should be ~1)")
print(f"  RMSE = {rmse:.4f} pH units")

# Buffer range
print("\nBuffer Properties:")
print(f"  Effective buffer range: {pKa_fit - 1:.2f} - {pKa_fit + 1:.2f} (pKa +/- 1)")

In [ ]:
# Calculate first derivative for inflection point
dV = V_titrant[1] - V_titrant[0]
dpH_dV = np.gradient(pH_measured, dV)
inflection_idx = np.argmax(dpH_dV)
Ve_inflection = V_titrant[inflection_idx]
pH_inflection = pH_measured[inflection_idx]

print("\nInflection Point Analysis:")
print(f"  Volume at inflection = {Ve_inflection:.2f} mL")
print(f"  pH at inflection = {pH_inflection:.2f}")
print(f"  Max slope = {dpH_dV[inflection_idx]:.2f} pH/mL")

## Buffer Capacity Analysis

In [ ]:
print("\nBUFFER CAPACITY ANALYSIS")
print("-" * 50)

pH_range = np.linspace(3, 7, 200)
beta_fitted = buffer_capacity(pH_range, pKa_fit, C_acid_true)
beta_true = buffer_capacity(pH_range, pKa_true, C_acid_true)

max_beta_idx = np.argmax(beta_fitted)
pH_max_beta = pH_range[max_beta_idx]
max_beta = beta_fitted[max_beta_idx]

print(f"Maximum capacity at pH = {pH_max_beta:.2f} (should equal pKa = {pKa_fit:.2f})")
print(f"Maximum beta = {max_beta:.4f} mol/(L*pH)")
print(f"Effective buffer range: {pKa_fit - 1:.2f} - {pKa_fit + 1:.2f} (pKa +/- 1)")

## Visualization

In [ ]:
fig = plt.figure(figsize=(16, 10))

# Plot 1: Monoprotic titration curve
ax1 = plt.subplot(2, 3, 1)
ax1.errorbar(
    V_titrant,
    pH_measured,
    yerr=sigma_pH,
    fmt="o",
    markersize=4,
    alpha=0.6,
    label="Measured pH",
    capsize=2,
)
ax1.plot(V_fit, pH_fitted_curve, "r-", linewidth=2, label="Fitted curve (robust)")
ax1.axvline(Ve_fit, color="g", linestyle="--", label=f"Ve = {Ve_fit:.2f} mL")
ax1.axhline(pKa_fit, color="orange", linestyle="--", alpha=0.5, label=f"pKa = {pKa_fit:.2f}")

ax1.set_xlabel("Volume of NaOH (mL)", fontsize=11)
ax1.set_ylabel("pH", fontsize=11)
ax1.set_title("Monoprotic Titration - fit() API", fontsize=12, fontweight="bold")
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# Plot 2: First derivative (slope)
ax2 = plt.subplot(2, 3, 2)
ax2.plot(V_titrant, dpH_dV, "b-", linewidth=2)
ax2.axvline(
    Ve_inflection,
    color="r",
    linestyle="--",
    label=f"Inflection: {Ve_inflection:.2f} mL",
)

ax2.set_xlabel("Volume of NaOH (mL)", fontsize=11)
ax2.set_ylabel("dpH/dV (pH/mL)", fontsize=11)
ax2.set_title("First Derivative\n(Equivalence Point Detection)", fontsize=12, fontweight="bold")
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

# Plot 3: Residuals
ax3 = plt.subplot(2, 3, 3)
ax3.scatter(V_fit, residuals, alpha=0.6, s=30)
ax3.axhline(0, color="r", linestyle="--", linewidth=1)
ax3.axhline(2 * rmse, color="orange", linestyle=":", label=f"+/- 2sigma ({2 * rmse:.3f})")
ax3.axhline(-2 * rmse, color="orange", linestyle=":")

ax3.set_xlabel("Volume of NaOH (mL)", fontsize=11)
ax3.set_ylabel("Residuals (pH units)", fontsize=11)
ax3.set_title(f"Residuals (RMSE = {rmse:.4f})", fontsize=12, fontweight="bold")
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3)

# Plot 4: Buffer capacity
ax4 = plt.subplot(2, 3, 4)
ax4.plot(pH_range, beta_fitted, "b-", linewidth=2, label="Fitted pKa")
ax4.plot(pH_range, beta_true, "r--", linewidth=2, alpha=0.5, label="True pKa")
ax4.axvline(pKa_fit, color="g", linestyle="--", alpha=0.5, label=f"pKa = {pKa_fit:.2f}")
ax4.axvline(pKa_fit - 1, color="orange", linestyle=":", alpha=0.5)
ax4.axvline(pKa_fit + 1, color="orange", linestyle=":", alpha=0.5, label="pKa +/- 1")

ax4.set_xlabel("pH", fontsize=11)
ax4.set_ylabel("Buffer Capacity beta (mol/L*pH)", fontsize=11)
ax4.set_title("Buffer Capacity vs pH", fontsize=12, fontweight="bold")
ax4.legend(fontsize=9)
ax4.grid(True, alpha=0.3)

# Plot 5: Parameter comparison
ax5 = plt.subplot(2, 3, 5)
params_names = ["pKa", "Ve (mL)"]
params_true_vals = [pKa_true, Ve_true]
params_fitted = [pKa_fit, Ve_fit]
params_err = [pKa_err, Ve_err]

x_pos = np.arange(len(params_names))
width = 0.35

ax5.bar(x_pos - width / 2, params_true_vals, width, label="True", alpha=0.7, color="blue")
ax5.bar(
    x_pos + width / 2,
    params_fitted,
    width,
    yerr=params_err,
    label="Fitted",
    alpha=0.7,
    color="red",
    capsize=5,
)

ax5.set_xticks(x_pos)
ax5.set_xticklabels(params_names)
ax5.set_ylabel("Value", fontsize=11)
ax5.set_title("Parameter Recovery", fontsize=12, fontweight="bold")
ax5.legend(fontsize=9)
ax5.grid(True, alpha=0.3, axis="y")

# Plot 6: API comparison
ax6 = plt.subplot(2, 3, 6)
ax6.axis("off")

summary_text = [
    ["Method", "pKa", "Ve (mL)"],
    ["-" * 20, "-" * 8, "-" * 10],
    ["fit() 'robust'", f"{pKa_fit:.3f}", f"{Ve_fit:.2f}"],
    ["fit() 'global'", f"{pKa_g:.3f}", f"{Ve_g:.2f}"],
    ["fit() custom", f"{pKa_c:.3f}", f"{Ve_c:.2f}"],
    ["", "", ""],
    ["True values", f"{pKa_true:.3f}", f"{Ve_true:.2f}"],
    ["", "", ""],
    ["Advantages of fit():", "", ""],
    ["  - Multi-start opt.", "", ""],
    ["  - Global search", "", ""],
]

table_text = "\n".join(["  ".join(row) for row in summary_text])
ax6.text(
    0.1,
    0.9,
    table_text,
    fontsize=10,
    verticalalignment="top",
    fontfamily="monospace",
    transform=ax6.transAxes,
)
ax6.set_title("fit() API Comparison", fontsize=12, fontweight="bold")

plt.tight_layout()
plt.savefig("titration_curves_analysis.png", dpi=300, bbox_inches="tight")
plt.show()

## Summary

In [ ]:
print("=" * 60)
print("SUMMARY: Acid-Base Titration Analysis")
print("=" * 60)

print("\n1. MONOPROTIC TITRATION (Acetic Acid):")
print(f"   pKa (fitted) = {pKa_fit:.3f} +/- {pKa_err:.3f}")
print(f"   pKa (true) = {pKa_true:.2f}")
print(f"   Equivalence point = {Ve_fit:.2f} +/- {Ve_err:.2f} mL")
print(f"   Buffer range = {pKa_fit - 1:.2f} - {pKa_fit + 1:.2f} (pKa +/- 1)")
print(f"   Max buffer capacity = {max_beta:.4f} mol/(L*pH) at pH = {pH_max_beta:.2f}")
print(f"   Fit quality: RMSE = {rmse:.4f}, chi^2/dof = {reduced_chi_squared:.3f}")

print("\n2. API Methods Used:")
print("   - fit() with preset='robust' (5 multi-starts)")
print("   - fit() with preset='global' (20 multi-starts)")
print("   - fit() with GlobalOptimizationConfig (custom settings)")

print("\n3. KEY INSIGHTS:")
print("   - Henderson-Hasselbalch equation accurately models buffer region")
print("   - Equivalence points identified from inflection points (max dpH/dV)")
print("   - Buffer capacity maximized at pH = pKa")
print("   - fit() API provides robust parameter estimation")